In [1]:
import re

input_file = "input_files/day_12.txt"

with open(input_file) as lines:
    data = [l.strip() for l in lines]

In [2]:
rx = re.compile(r"#+")

def parse_lines(data):
    lines = []
    for line in data:
        pattern, counts = line.split()
        counts = tuple([int(n) for n in counts.split(',')])
        lines.append((pattern, counts))
    return lines
    
lines = parse_lines(data)   
len(lines), lines[0]

(1000, ('.?????...?', (1, 1, 1)))

# Part One

The basic idea is to consume each count by matching a string like ".###." (if count is three) onto the pattern. For each match recurse on the rest of the pattern and remaining counts. Part two is addressed by memoizing with the LRU cache. No doubt, there is a simpler way.

In [3]:
from functools import cache

@cache
def find_matches(sub, counts):
    if len(counts) == 0:
        return '#' not in sub
        
    min_string = sum(counts) + len(counts) + 1 

    n, *counts = counts

    matcher = '.' + '#' * n + '.'
    found = 0
    
    for i in range(len(sub) - (min_string - n) ):
        test_string = sub[i:i+n+2]
        remaining = '.' + sub[i+n+2:]

        if test_string[1:1+n] == '#' * n:
            if test_string[-1] != "#":
                found += find_matches(remaining, tuple(counts))
            break
        else:
            if sub[i] == '#':
                break
            if all(b=='?' or a==b for a, b in zip(matcher, sub[i:i+n+2])):
                remaining = '.' + sub[i+n+2:]
                found += find_matches(remaining, tuple(counts))
                
    return found

In [4]:
sum(find_matches('.'+match[0]+'.', tuple(match[1])) for match in lines)

7922

# Part Two

In [5]:
def unfold(pattern, counts):
    return ('.'+'?'.join([pattern] * 5) + '.', counts * 5)

sum(find_matches(*(unfold(*match))) for match in lines)


18093821750095